In [1]:
import pandas as pd
import numpy as np
import re
import os
import json
from openpyxl import load_workbook

In [2]:
# Load all the necessary Excel files to notebook

#file for contractor group
first_df = pd.read_excel('storti data.xlsx',sheet_name ='Sheet1')

address_df = pd.read_excel('recon_addresses.xlsx',sheet_name='Sheet1') # addresses for contractor groups

#dictionary for address parsing (street names and arteries)
street_mapping_df = pd.read_excel('Copy of Dictionary master.xlsx', sheet_name='Street Names') 
artery_mapping_df = pd.read_excel('Copy of Dictionary master.xlsx', sheet_name='Art')

#file with default values and data standardization 
mapping_df = pd.read_excel('mapping2.xlsx',sheet_name = 'STORTI')

#blank template to convert to
new_df = pd.read_excel('SLI_Template.xlsx')

with open('value_mapping.json', 'r') as f:
    value_mapping = json.load(f)

In [ ]:
file_to_read = input("Which database are you trying to run? \n ---> RECONN \n ---> SLI \n ---> STORTI \n")

if file_to_read == "RECONN":
    first_df[['corrosion_found_on_select_all_that_apply2', 'corrosion_location_select_all_that_apply','is_coned_piping_accessible','not_accessible_reason2','poesleeve_location','inspected_all_services_entering_building','any_impropersubstandard_conditions','corrosion_severity','corrosion_location_select_all_that_apply','do_you_have_access_to_the_gas_service_point_of_entry_poe','no_access_reason','is_the_gas_poe_sealed','how_did_you_seal_it']].replace('0','',inplace=True)
    first_df.iloc[:,1:] = first_df.iloc[:,1:].applymap(lambda x:x.upper() if isinstance(x,str) else x)
    # first_df["meter_location_atmosphere_select_all_that_apply"] = first_df["meter_location_atmosphere_select_all_that_apply"].str.split(';').str[0]
elif file_to_read == 'SLI':
    first_df[['Corrosion found on?', 'Corrosion location? ','Any improper or substandard conditions?','Was the Con Edison pipe/equipment accessible?','No Access Reason2','Point of Entry (P.O.E) sleeve location?','Inspection Done on all Services Entering the Bldg?','Any Improper or Substandard Conditions?2','Corrosion severity on Con Ed piping?','Corrosion location? 2','Did you have access to Gas Service POE (point of entry)?','Reason Not Accessible','Gas Service Point of Entry Found Sealed?','How did you seal it?']].replace('0','',inplace=True)    
    first_df.iloc[:,1:] = first_df.iloc[:,1:].applymap(lambda x:x.upper() if isinstance(x,str) else x)    
    first_df["Meter Location Atmosphere "] = first_df["Meter Location Atmosphere "].str.split(',').str[0]
elif file_to_read == "STORTI":
    first_df[['Meter Location','Service Under Stoop','Meter Location Atmosphere 1']].replace('0','',inplace=True)
    first_df.iloc[:,1:] = first_df.iloc[:,1:].applymap(lambda x:x.upper() if isinstance(x,str) else x)    
else:
    raise ValueError("Invalid input, type exactly as options state")

# create a dictionary from the mapping_df file
mapping_dict = mapping_df.set_index('normalized name')['column name'].to_dict()
default_values_dict = mapping_df.set_index('normalized name')['default value'].apply(lambda x: x.split(',') if isinstance(x, str) else x).to_dict()
new_df = pd.DataFrame(columns = new_df.columns)

In [ ]:
for column in new_df.columns:
    if column in mapping_dict and mapping_dict[column] in first_df.columns:
        column_name = mapping_dict[column]
        new_df[column] = first_df[column_name]
    else:
        new_df[column]=new_df[column]

In [ ]:
BuildingAccessible_map = value_mapping.get('BuildingAccessible',{})
OutsideSet_map = value_mapping.get('OutsideSet',{})
PressureRegulatorInBuilding_map = value_mapping.get('PressureRegulatorInBuilding',{})
TypeOfDwellingCd_map = value_mapping.get('TypeOfDwellingCd',{})
MeterLocation_map = value_mapping.get('MeterLocation',{})
MeterLocationAtmosphere_map = value_mapping.get('MeterLocationAtmosphere',{})
PipingConstructionMaterialCd_map = value_mapping.get('PipingConstructionMaterialCd',{})
MeterConstructionMaterialCd_map = value_mapping.get('MeterConstructionMaterialCd',{})
FittingsConstructionMaterialCd_map = value_mapping.get('FittingsConstructionMaterialCd',{})
MainAndBranchService_map = value_mapping.get('MainAndBranchService',{})
NoAccessReasonCdBuilding_map = value_mapping.get('NoAccessReasonCdBuilding',{})
ServiceUnderStoop_map = value_mapping.get('ServiceUnderStoop',{})
AccessibleCustomer_map = value_mapping.get('AccessibleCustomer',{})
NoAccessReasonCdCustomer_map = value_mapping.get('NoAccessReasonCdCustomer',{})
CorrosionSeverityCdCustomer_map = value_mapping.get('CorrosionSeverityCdCustomer',{})
CorrosionLocationsCdCustomer_map = value_mapping.get('CorrosionLocationsCdCustomer',{})
CtagEquipmentCd_map = value_mapping.get('CtagEquipmentCd',{})
HeadOfServiceValveNotAccessibleHeight_map = value_mapping.get('HeadOfServiceValveNotAccessibleHeight',{})
GasTurnedOff_map = value_mapping.get('GasTurnedOff',{})
AllServicePipingInspected_map = value_mapping.get('AllServicePipingInspected',{})
CorrosionFoundOnCdCustomer_map = value_mapping.get('CorrosionFoundOnCdCustomer',{})
ImproperConditionsFoundCustomer_map = value_mapping.get('ImproperConditionsFoundCustomer',{})
DamagedGasMeterCustomerCd_map = value_mapping.get('DamagedGasMeterCustomerCd',{})
InappropriateFlexibleConnectorCds_map = value_mapping.get('InappropriateFlexibleConnectorCds',{})
AccessibleConEd_map = value_mapping.get('AccessibleConEd',{})
NoAccessReasonCdConed_map = value_mapping.get('NoAccessReasonCdConed',{})
ImproperSubstandardConditionCustomerCds_map = value_mapping.get('ImproperSubstandardConditionCustomerCds',{})
POESleeveLocationCd_map = value_mapping.get('POESleeveLocationCd',{})
InspDoneOnAllSvcsEnteringBldg_map = value_mapping.get('InspDoneOnAllSvcsEnteringBldg',{})
ImproperConditionsFoundConEd_map = value_mapping.get('ImproperConditionsFoundConEd',{})
ImproperSubstandardConditionConEdCds_map = value_mapping.get('ImproperSubstandardConditionConEdCds',{})
DamagedGasMeterConEdCd_map = value_mapping.get('DamagedGasMeterConEdCd',{})
UnauthorizedMetersFound_map = value_mapping.get('UnauthorizedMetersFound',{})
SupervisorNotified_map = value_mapping.get('SupervisorNotified',{})
PlasticExposed_map = value_mapping.get('PlasticExposed',{})
PlasticExtendsIntoStructure24Inches_map = value_mapping.get('PlasticExtendsIntoStructure24Inches',{})
CorrosionSeverityCdConEd_map = value_mapping.get('CorrosionSeverityCdConEd',{})
CorrosionFoundOnCdConEd_map = value_mapping.get('CorrosionFoundOnCdConEd',{})
CorrosionLocationsCdConEd_map = value_mapping.get('CorrosionLocationsCdConEd',{})
NGDRemovedReasonCd_map = value_mapping.get('NGDRemovedReasonCd',{})
PoeAccessible_map = value_mapping.get('PoeAccessible',{})
PoeNoAccessReasonCd_map = value_mapping.get('PoeNoAccessReasonCd',{})
PoeFoundSealedCondition_map = value_mapping.get('PoeFoundSealedCondition',{})
PoeLeftSealedCondition_map = value_mapping.get('PoeLeftSealedCondition',{})
PoeSealTypeCd_map = value_mapping.get('PoeSealTypeCd',{})
GasOdorPresent_map = value_mapping.get('GasOdorPresent',{})
GasLeakCorrosionRelated_map = value_mapping.get('GasLeakCorrosionRelated',{})
LeakItemCd_map = value_mapping.get('LeakItemCd',{})

new_df['BuildingAccessible'] = new_df['BuildingAccessible'].map(BuildingAccessible_map).fillna('YES')
new_df['OutsideSet'] = new_df['OutsideSet'].map(OutsideSet_map).fillna('NO')
new_df['PressureRegulatorInBuilding'] = new_df['PressureRegulatorInBuilding'].map(PressureRegulatorInBuilding_map).fillna('YES')
new_df['TypeOfDwellingCd'] = new_df['TypeOfDwellingCd'].map(TypeOfDwellingCd_map).fillna('SINGLE')
new_df['MeterLocation'] = new_df['MeterLocation'].map(MeterLocation_map).fillna('METERROOM')
new_df['MeterLocationAtmosphere'] = new_df['MeterLocationAtmosphere'].map(MeterLocationAtmosphere_map).fillna('METERRROOM')
new_df['PipingConstructionMaterialCd'] = new_df['PipingConstructionMaterialCd'].map(PipingConstructionMaterialCd_map).fillna('METALLICBARE')
new_df['MeterConstructionMaterialCd'] = new_df['MeterConstructionMaterialCd'].map(MeterConstructionMaterialCd_map).fillna('METALLICBARE')
new_df['FittingsConstructionMaterialCd'] = new_df['FittingsConstructionMaterialCd'].map(FittingsConstructionMaterialCd_map).fillna('METALLICBARE')
new_df['MainAndBranchService'] = new_df['MainAndBranchService'].map(MainAndBranchService_map).fillna('YES')
# new_df['NoAccessReasonCdBuilding'] = new_df['NoAccessReasonCdBuilding'].map(NoAccessReasonCdBuilding_map).fillna('OTHER')
new_df['ServiceUnderStoop'] = new_df['ServiceUnderStoop'].map(ServiceUnderStoop_map).fillna('YES')
new_df['AccessibleCustomer'] = new_df['AccessibleCustomer'].map(AccessibleCustomer_map).fillna('YES')
new_df['NoAccessReasonCdCustomer'] = new_df['NoAccessReasonCdCustomer'].map(NoAccessReasonCdCustomer_map).fillna('OTHER')
new_df['CorrosionSeverityCdCustomer'] = new_df['CorrosionSeverityCdCustomer'].map(CorrosionSeverityCdCustomer_map).fillna('NONE')
new_df['CorrosionLocationsCdCustomer'] = new_df['CorrosionLocationsCdCustomer'].map(CorrosionLocationsCdCustomer_map).fillna('NONE')
new_df['CtagEquipmentCd'] = new_df['CtagEquipmentCd'].map(CtagEquipmentCd_map).fillna('')
# new_df['HeadOfServiceValveNotAccessibleHeight'] = new_df['HeadOfServiceValveNotAccessibleHeight'].map(HeadOfServiceValveNotAccessibleHeight_map).fillna('YES')
# new_df['GasTurnedOff'] = new_df['GasTurnedOff'].map(GasTurnedOff_map).fillna('NO')
new_df['GasTurnedOff'] = new_df['GasTurnedOff'].map(GasTurnedOff_map).fillna('NO')
new_df['AllServicePipingInspected'] = new_df['AllServicePipingInspected'].map(AllServicePipingInspected_map).fillna('YES')
new_df['CorrosionFoundOnCdCustomer'] = new_df['CorrosionFoundOnCdCustomer'].map(CorrosionFoundOnCdCustomer_map).fillna('OTHER')
# new_df['ImproperConditionsFoundCustomer'] = new_df['ImproperConditionsFoundCustomer'].map(ImproperConditionsFoundCustomer_map).fillna('NO')
# new_df['DamagedGasMeterCustomerCd'] = new_df['DamagedGasMeterCustomerCd'].map(DamagedGasMeterCustomerCd_map).fillna('OTHER')
# new_df['InappropriateFlexibleConnectorCds'] = new_df['InappropriateFlexibleConnectorCds'].map(InappropriateFlexibleConnectorCds_map).fillna('')
new_df['AccessibleConEd'] = new_df['AccessibleConEd'].map(AccessibleConEd_map).fillna('YES')
new_df['NoAccessReasonCdConed'] = new_df['NoAccessReasonCdConed'].map(NoAccessReasonCdConed_map).fillna('DIDNOTATTEMPT')
new_df['POESleeveLocationCd'] = new_df['POESleeveLocationCd'].map(POESleeveLocationCd_map).fillna('OTHER')
new_df['InspDoneOnAllSvcsEnteringBldg'] = new_df['InspDoneOnAllSvcsEnteringBldg'].map(InspDoneOnAllSvcsEnteringBldg_map).fillna('YES')
new_df['ImproperConditionsFoundConEd'] = new_df['ImproperConditionsFoundConEd'].map(ImproperConditionsFoundConEd_map).fillna('NO')
new_df['ImproperSubstandardConditionConEdCds'] = new_df['ImproperSubstandardConditionConEdCds'].map(ImproperSubstandardConditionConEdCds_map).fillna('OTHER')
# new_df['DamagedGasMeterConEdCd'] = new_df['DamagedGasMeterConEdCd'].map(DamagedGasMeterConEdCd_map).fillna('OTHER')
new_df['UnauthorizedMetersFound'] = new_df['UnauthorizedMetersFound'].map(UnauthorizedMetersFound_map).fillna('NO')
new_df['SupervisorNotified'] = new_df['SupervisorNotified'].map(SupervisorNotified_map).fillna('')
new_df['PlasticExposed'] = new_df['PlasticExposed'].map(PlasticExposed_map).fillna('')
new_df['PlasticExtendsIntoStructure24Inches'] = new_df['PlasticExtendsIntoStructure24Inches'].map(PlasticExtendsIntoStructure24Inches_map).fillna('')
new_df['CorrosionSeverityCdConEd'] = new_df['CorrosionSeverityCdConEd'].map(CorrosionSeverityCdConEd_map).fillna('NONE')
new_df['CorrosionFoundOnCdConEd'] = new_df['CorrosionFoundOnCdConEd'].map(CorrosionFoundOnCdConEd_map).fillna('OTHER')
new_df['CorrosionLocationsCdConEd'] = new_df['CorrosionLocationsCdConEd'].map(CorrosionLocationsCdConEd_map).fillna('OTHER')
# new_df['NGDRemovedReasonCd'] = new_df['NGDRemovedReasonCd'].map(NGDRemovedReasonCd_map).fillna('OTHER')
new_df['ImproperSubstandardConditionCustomerCds'] = new_df['ImproperSubstandardConditionCustomerCds'].map(ImproperSubstandardConditionCustomerCds_map).fillna('YES')
new_df['PoeAccessible'] = new_df['PoeAccessible'].map(PoeAccessible_map).fillna('YES')
new_df['PoeNoAccessReasonCd'] = new_df['PoeNoAccessReasonCd'].map(PoeNoAccessReasonCd_map).fillna('OTHER')
new_df['PoeFoundSealedCondition'] = new_df['PoeFoundSealedCondition'].map(PoeFoundSealedCondition_map).fillna('NO')
new_df['PoeLeftSealedCondition'] = new_df['PoeLeftSealedCondition'].map(PoeLeftSealedCondition_map).fillna('NO')
new_df['PoeSealTypeCd'] = new_df['PoeSealTypeCd'].map(PoeLeftSealedCondition_map).fillna('OTHER')
new_df['PoeSealTypeCd'] = new_df['PoeSealTypeCd'].fillna('OTHER')
new_df['GasOdorPresent'] = new_df['GasOdorPresent'].map(GasOdorPresent_map).fillna('NO')
new_df['GasLeakCorrosionRelated'] = new_df['GasLeakCorrosionRelated'].map(GasLeakCorrosionRelated_map).fillna('NO')
new_df['LeakItemCd'] = new_df['LeakItemCd'].map(LeakItemCd_map).fillna('OTHER')

In [ ]:
# file_to_read = input("WHICH DATABASE ARE YOU RUNNING? \n ---> RECONN \n ---> SLI \n ---> STORTI \n")

if os.path.basename(file_to_read) == 'RECONN':
    print("RUNNING RECONN")
    new_df = new_df[first_df['outside_set']!='YES']
    new_df = new_df[first_df['accessible']!='NO']

    new_df['Borough'] = first_df['borough']
    new_df['CompletedBy'] = first_df['_assigned_to']
    new_df['StartedDateTime']=first_df['_server_cr']
    new_df['AssignedDateTime']=first_df['_server_updated_at']
    new_df['NoOfMeters']=first_df['number_of_meters']
    new_df['NoOfApartments']=first_df['number_of_apts']
    new_df['AssetRemarks']=first_df['remarks']
    new_df['GasLeakReadingCGI']=first_df['cgi_reading_gas']
    new_df['GasLeakTicketNbr']=first_df['leak_ticket_']
    new_df['LeakItemRemarks']=first_df['remarks5']
    new_df['HouseNumber'] = first_df['address_sub_thoroughfare']
    new_df['StreetName'] = first_df['address_thoroughfare']
    new_df['ZipCode'] = first_df['address_postal_code'] 
    
    
    # new_df['InspDoneOnAllSvcsEnteringBldg'] = new_df['InspDoneOnAllSvcsEnteringBldg'].apply(lambda x:'YES' if x == 0 or x == 'yes' else 'NO')
    # new_df['UnauthorizedMetersFound'] = new_df['UnauthorizedMetersFound'].apply(lambda x: 'YES' if x == 'yes' else 'NO' if x in [0, 'no'] else '')
    new_df['TypeOfDwellingCd'] = new_df.apply(lambda row: 'SINGLE' if row['TypeOfDwellingRemarks'] == 0 else 'MULTIUNIT' if row['NoOfMeters'] > 2 else TypeOfDwellingCd_map.get(row['TypeOfDwellingCd'],'OTHER') if row['TypeOfDwellingCd'] and pd.notna(row['TypeOfDwellingCd']) else 'SINGLE', axis=1)
    # new_df['TypeOfDwellingCd'] = new_df.apply(lambda row: 'MULTIUNIT' if row['NoOfMeters'] >= 2 else TypeOfDwellingCd_map.get(row['TypeOfDwellingCd'],'OTHER') if row['TypeOfDwellingCd'] and pd.notna(row['TypeOfDwellingCd']) else 'SINGLE', axis=1)
    new_df['PoeNoAccessReasonCd'] = new_df.apply(lambda row: '' if row['PoeAccessible'] == 'YES' else row['PoeNoAccessReasonCd'], axis = 1)
    new_df['NoAccessReasonCdCustomer'] = new_df.apply(lambda row: '' if row['AccessibleCustomer'] in ['YES', 0] else row['NoAccessReasonCdCustomer'], axis=1)
    
    for column, mapping in value_mapping.items():
        if column != 'WR#' and column != 'InspectionType' and column != 'Muni' and column !='StatedDateTime' and column != 'AssignedDateTime' and column != 'NoOfMeters' and column != 'NoOfApartments' and column != 'AssetRemarks' and column != 'GasLeakReadingCGI' and column != 'GasLeakTicketNbr' and column != 'LeakItemRemarks' and column != 'ServiceSizeCd' and column != 'ServiceMaterialCd' and column != 'InspectionDateTime' and column != 'MeterLocationRemarks' and column != 'MeterLocationAtmosphereRemarks' and column != 'NoAccessReasonCustomerRemarks' and column != 'CorrosionSeverityCdCustomer' and column != 'GasTurnedOff' and column != 'CorrosionLocationCustomerRemarks' and column != 'ImproperConditionsFoundCustomer' and column != 'HeadOfServiceValveNotAccessibleHeight' and column != 'DamagedGasMeterCustomerCd' and column != 'INAPPRGROUNDPIPINGCD' and column != 'InappropriateFlexibleConnectorCds' and column != 'ImproperSubstandardConditionsCustomerOtherDescribe' and column != 'CorrosionInspectionRemarks' and column != 'DamagedGasMeterConEdCd':
            new_df[column] = new_df[column].replace(mapping).fillna(mapping.get(np.nan,np.nan))
    
    new_df.to_excel('reconn_parsed.xlsx',index=False)
     
elif os.path.basename(file_to_read) == 'SLI':
    print("RUNNING SLI")
    
    new_df = new_df[first_df['All Meters Outdoors for Services Entering Building ']!='YES']
    new_df = new_df[first_df['Is the building accessible?']!='NO']
    
    new_df['CtagEquipmentCd'] = new_df['CtagEquipmentCd'].fillna('OES')

    new_df['InspectionDateTime'] = first_df['Date of the Inspection']
    new_df['StartedDateTime'] = first_df['Start time']
    new_df['HouseNumber'] = first_df['House Number']
    new_df['CompassPoint'] = first_df['Compass Point']
    new_df['StreetName'] = first_df['Street Name']
    new_df['Artery'] = first_df['Artery']
    new_df['ZipCode'] = first_df['Zipcode']
    new_df['Borough'] = first_df['Borough']
    new_df['Muni'] = first_df['Municipality']
    new_df['TypeOfDwellingRemarks'] = first_df['Number of Apartments']
    new_df['NoOfMeters'] = first_df['Number of Meters']
    new_df['NoOfApartments'] = first_df['Number of Apartments']
    new_df['AssetRemarks'] = first_df['Asset Remarks']
    new_df['CtagEquipmentRemarks'] = first_df['Tag equipment remarks']
    new_df['CorrosionInspectionRemarks'] = first_df['Remarks']
    new_df['RemarksConEd'] = first_df['General Remarks']
    new_df['PoeRemarks'] = first_df['Gas POE Remarks']
    new_df['GasLeakReadingCGI'] = first_df['C.G.I. Reading (Percent Gas)']
    new_df['GasLeakTicketNbr'] = first_df['Leak ticket number']
    new_df['LeakInspectionRemarks'] = first_df['Leak Inspection Remarks']
    new_df['Employee_No'] = first_df['Please Enter Inspector Employee ID']

    new_df['TypeOfDwellingCd'] = new_df['TypeOfDwellingCd'].apply(lambda x: TypeOfDwellingCd_map.get(str(x).lower(), TypeOfDwellingCd_map.get(x, 'OTHER')) if x and pd.notna(x) else 'SINGLE')
    # new_df['CorrosionSeverityCdCustomer'] = new_df['CorrosionSeverityCdCustomer'].astype(str).apply(lambda x: CorrosionSeverityCdCustomer_map.get(x.lower(), 'NONE'))
    # new_df['CorrosionSeverityCdCustomer'] = new_df['CorrosionSeverityCdCustomer'].apply(lambda x: CorrosionSeverityCdCustomer_map.get(x.lower(), 'NONE'))
    new_df['CtagEquipmentCd'] = new_df.apply(lambda row: CtagEquipmentCd_map.get(row['CorrosionSeverityCdCustomer'], row['CtagEquipmentCd']), axis=1)
    new_df['PipingConstructionMaterialCd'] = new_df['PipingConstructionMaterialCd'].apply(lambda x: PipingConstructionMaterialCd_map.get(x, 'OTHER') if x and pd.notna(x) else 'METALLICBARE')
    new_df['MeterConstructionMaterialCd'] = new_df['MeterConstructionMaterialCd'].apply(lambda x: MeterConstructionMaterialCd_map.get(x, 'OTHER') if x and pd.notna(x) else 'METALLICBARE')
    new_df['FittingsConstructionMaterialCd'] = new_df['FittingsConstructionMaterialCd'].apply(lambda x: FittingsConstructionMaterialCd_map.get(x, 'OTHER') if x and pd.notna(x) else 'METALLICBARE')
    new_df['MainAndBranchService'] = new_df['MainAndBranchService'].apply(lambda x: MainAndBranchService_map.get(x, 'OTHER') if x and pd.notna(x) else 'YES')
    new_df['NoOfMeters'] = new_df.apply(lambda row: 1 if row['TypeOfDwellingCd'] == 'SINGLE' else row['NoOfMeters'], axis=1)
    new_df['NoOfApartments'] = new_df.apply(lambda row: '' if row['TypeOfDwellingCd'] == 'SINGLE' else row['NoOfApartments'], axis=1)

    for column, mapping in value_mapping.items():
        if column!='TypeOfDwellingCd' and column!='MeterLocation' and column!='PipingConstructionMaterialCd' and column!='MeterConstructionMaterialCd' and column!='MainAndBranchService' and column!='FittingsConstructionMaterialCd':
            new_df[column] = new_df[column].replace(mapping)
            
    new_df.to_excel('sli_parsed.xlsx',index=False)
    
elif os.path.basename(file_to_read) == 'STORTI':
    print("Running Storti")
    new_df = new_df[first_df['Outside Set']!='YES']
    new_df = new_df[first_df['Building Accessible']!='NO']
    
    new_df['InspectionDateTime'] = first_df['Inspection Date']
    new_df['CompletedBy'] = first_df['Employee Name']
    new_df['ZipCode'] = first_df['ZipCode']
    new_df['Borough'] = first_df['Borough']
    new_df['MeterLocationRemarks'] = first_df['Meter Location Remarks']
    new_df['AssetRemarks'] = first_df['Asset Remarks']
    new_df['CorrosionFoundOnCustomerRemarks'] = first_df['Corrosion Inspection Remarks']
    new_df['CorrosionInspectionRemarks'] = first_df['Corrosion Inspection Remarks']
    new_df['NoAccessReasonConedRemarks'] = first_df['No Access Reason ConEd Remarks']
    new_df['POESleeveLocationRemarks'] = first_df['POE Sleeve Location Remarks']
    new_df['ImproperSubstandardConditionsConEdOtherDescribe'] = first_df['Improper Substandard Conditions ConEd Other Describe']
    new_df['CorrosionFoundOnConEdRemarks'] = first_df['Corrosion Found On ConEd Remarks']
    new_df['CorrosionLocationConEdRemarks'] = first_df['Corrosion Location ConEd Remarks']
    new_df['RemarksConEd'] = first_df['Remarks ConEd']
    new_df['PoeNoAccessReasonRemarks'] = first_df['POE No Access Reason Remarks']
    new_df['PoeSealTypeRemarks'] = first_df['POE Seal Type Remarks']
    new_df['PoeRemarks'] = first_df['POE Remarks']
    new_df['GasLeakReadingCGI'] = first_df['Gas Leak Reading CGI']
    new_df['GasLeakTicketNbr'] = first_df['Gas Leak Ticket Nbr']
    new_df['LeakInspectionRemarks'] = first_df['Leak Inspection Remarks']
    new_df['Employee_No'] = first_df['Employee ID']
    
    for column, mapping in value_mapping.items():
        new_df[column] = new_df[column].replace(mapping)
    
    new_df.to_excel('storti_parsed.xlsx',index=False)
      
else:
    raise ValueError("Your input is incorrect, please enter one of the three again.")
    
print("Finished running")

In [7]:
# print(first_df['pressure_regulator_in_building'].unique())  # Print unique values in new_df['GasTurnedOff']
# print(new_df['OutsideSet'].isna().sum())  # Print number of NaN values in new_df['GasTurnedOff']
for columns in first_df:
    
    print(first_df[columns])

0          2S
1          3S
2          4S
3          5S
4          6S
        ...  
2640    2642S
2641    2643S
2642    2644S
2643    2645S
2644    2646S
Name: ID, Length: 2645, dtype: object
0      2023-01-03
1      2023-01-03
2      2023-01-03
3      2023-01-03
4      2023-01-03
          ...    
2640   2023-11-14
2641   2023-11-11
2642   2023-11-14
2643   2023-11-14
2644   2023-11-14
Name: Inspection Date, Length: 2645, dtype: datetime64[ns]
0        WR000000000
1         0000000000
2           00000000
3       WR0000000000
4        WR100290906
            ...     
2640       100691153
2641       100455639
2642    WR 100257389
2643       100447022
2644    WR 100257547
Name: WR Number, Length: 2645, dtype: object
0               Daniel Soto
1            Anderson Nunez
2       Michael Oluwamuleya
3             Cristian Alas
4           Bryant Mercedes
               ...         
2640              Ali Emadi
2641              Ali Emadi
2642         Oscar Delacruz
2643          Dwight Sp